### 蒙特卡罗方法 MC Basic方法
主要思想：

In [ ]:
import gymnasium as gym
import numpy as np
import time

# 创建环境（训练时不渲染，加快速度）
env = gym.make("FrozenLake-v1", is_slippery=False)

n_states = env.observation_space.n      # 16
n_actions = env.action_space.n          # 4

# 初始化策略为均匀随机策略
policy = np.ones((n_states, n_actions)) / n_actions

# 初始化 Q 表和 returns 字典
Q = np.zeros((n_states, n_actions))
returns = {(s, a): [] for s in range(n_states) for a in range(n_actions)}

# 超参数
num_episodes_for_evaluation = 20000
gamma = 1.0

print("正在使用随机策略生成 episodes 并评估 Q(s,a)...")

# 策略评估,使用当前策略（随机）生成 episodes，计算 Q
for _ in range(num_episodes_for_evaluation):
    episode = []
    state, _ = env.reset()
    done = False

    # 用当前策略生成一个完整episode
    while not done:
        action_probs = policy[state]
        action = np.random.choice(n_actions, p=action_probs)
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        done = terminated or truncated

    # 计算每个 (s,a) 的 first-visit return 并记录
    visited = set()
    G = 0
    # 从 episode 末尾反向计算 return,G(t) = R0 + γR1 + γ^2R2 ....
    for t in reversed(range(len(episode))):
        state, action, reward = episode[t]
        G = gamma * G + reward
        if (state, action) not in visited:
            visited.add((state, action))
            returns[(state, action)].append(G)

# 更新 Q 表：对每个 (s,a)，取所有 observed return 的平均值
for s in range(n_states):
    for a in range(n_actions):
        if returns[(s, a)]:  # 避免除零或空列表
            Q[s][a] = np.mean(returns[(s, a)])


print("Q 表评估完成。")

# 策略改进 —— 对每个状态选择 Q 最大的动作（greedy）
new_policy = np.zeros((n_states, n_actions))
for s in range(n_states):
    best_action = np.argmax(Q[s])
    new_policy[s] = np.eye(n_actions)[best_action]

print("策略改进完成。新策略为确定性 greedy 策略。")
print("每个状态选择的动作（0=左,1=下,2=右,3=上）:")
deterministic_actions = np.argmax(new_policy, axis=1)
print(deterministic_actions)

# Step 5: 可视化测试新策略
print("\n正在用新策略运行一个可视化 episode...")
env_test = gym.make("FrozenLake-v1", is_slippery=False, render_mode='human')
state, _ = env_test.reset()
done = False
total_reward = 0

while not done:
    action = deterministic_actions[state]
    state, reward, terminated, truncated, _ = env_test.step(action)
    total_reward += reward
    done = terminated or truncated
    time.sleep(1)

print(f"测试 episode 总奖励: {total_reward}")
env_test.close()
env.close()

### 蒙特卡罗探索 MC Exploring Starts 算法
伪代码：
初始化 Q(s,a) 任意，π(s) 任意
repeat:
   1. 随机选择一个 (s₀, a₀) 作为 episode 起点（exploring starts）
   2. 从 s₀ 执行 a₀，然后按当前策略 π 生成完整 episode
   3. 计算该 episode 中每个首次访问的 (s,a) 的 return G
   4. 对这些 (s,a)，更新 Q(s,a) ← average of all observed returns so far
       （可以用 incremental mean: Q ← Q + (G - Q)/N
   5. 对所有 s，设置 π(s) ← argmax_a Q(s,a)   ← 立即 greedy 改进！
until Q 收敛

In [ ]:
import gymnasium as gym
import numpy as np
import time
env = gym.make("FrozenLake-v1", is_slippery=False)
nS = env.observation_space.n
nA = env.action_space.n
desc = env.unwrapped.desc
n_row, n_col = desc.shape

valid_states = [s for s in range(nS) if desc[s//n_col][s%n_col] in (b'F', b'S')]

# 初始化
Q = np.zeros((nS, nA))
# 初始策略：随机
policy = np.ones((nS, nA)) / nA

N = np.zeros((nS, nA))  # 访问次数

num_episodes = 20000
gamma = 1.0

for i in range(num_episodes):
    # 随机选 (s0, a0)
    s0 = np.random.choice(valid_states)
    a0 = np.random.choice(nA)

    env.reset()
    env.unwrapped.s = s0
    s, r, terminated, truncated, _ = env.step(a0)
    episode = [(s0, a0, r)]

    # 后续按当前策略走
    while not (terminated or truncated):
        probs = policy[s]
        a = np.random.choice(nA, p=probs)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode.append((s, a, r))
        s = s_next

    # 计算 return 并更新 Q
    G = 0
    visited = set()
    for t in reversed(range(len(episode))):
        s, a, r = episode[t]
        G = gamma * G + r
        if (s, a) not in visited:
            visited.add((s, a))
            N[s][a] += 1
            Q[s][a] += (G - Q[s][a]) / N[s][a]

    # 立即策略改进（greedy）
    for s in range(nS):
        best_a = np.argmax(Q[s])
        policy[s] = 0
        policy[s][best_a] = 1.0

print("策略改进完成。")
print("每个状态选择的动作（0=左,1=下,2=右,3=上）:")
deterministic_actions = np.argmax(policy, axis=1)
print(deterministic_actions)

# 可视化测试新策略
print("\n正在用新策略运行一个可视化 episode...")
env_test = gym.make("FrozenLake-v1", is_slippery=False, render_mode='human')
state, _ = env_test.reset()
done = False
total_reward = 0

while not done:
    action = deterministic_actions[state]
    state, reward, terminated, truncated, _ = env_test.step(action)
    total_reward += reward
    done = terminated or truncated
    time.sleep(1)

print(f"测试 episode 总奖励: {total_reward}")
env_test.close()
env.close()

### MC ε-Greedy算法

In [39]:
import numpy as np
import time
from rl_utils.GNGridWorldEnv import GridWorldEnv

# ε-Greedy策略生成
def epsilon_greedy_action(q_table, state, epsilon, n_actions):
    if np.random.rand() < epsilon:
        return np.random.randint(0, n_actions)
    else:
        return np.argmax(q_table[state])

# 设置环境
grid_world_size = 8
env = GridWorldEnv(size=grid_world_size)
n_actions = env.action_space.n
q_table = np.zeros((grid_world_size * grid_world_size, n_actions))

env.render()  # 显示窗口


# 初始参数
gamma = 0.95
epsilon = 1.0   # 随机epsilon，强探索
epsilon_decay = 0.995   #eps的每轮衰减率
epsilon_min = 0.01 # 最小的eps
max_iterate = 1000

state_count = np.zeros((grid_world_size * grid_world_size, n_actions))  # 访问次数
for it_index in range(max_iterate):

    observation, _ = env.reset(None, options={'enable_random_pos': True})
    state = int((observation['agent'][0] * grid_world_size) + observation['agent'][1])
    done = False
    episode_max = 200
    episode = []
    while not done and episode_max >0:
        action = epsilon_greedy_action(q_table, state, epsilon, n_actions)
        next_state, reward, terminated, truncated, _ = env.step(action)
        episode.append((state, action, reward))
        state = int((next_state['agent'][0] * grid_world_size) + next_state['agent'][1])
        done = terminated or truncated
        episode_max -= 1

    # 用平均回报更新Q表
    G = 0.0
    for t in reversed(range(len(episode))):
        s, a, r = episode[t]
        state_count[s][a] += 1
        G = r + gamma * G
        n = state_count[s][a]
        q_table[s][a] += (1.0 / state_count[s][a]) * (G - q_table[s][a])

    # 更新epsilon
    epsilon = max(epsilon * epsilon_decay,epsilon_min)
    # 可选：打印进度
    if it_index % 100 == 0:
        print(f"Episode {it_index}, epsilon: {epsilon:.4f}")


print("\n正在用新策略运行一个可视化 episode...")
test_env = GridWorldEnv(size=grid_world_size, render_mode='human')
n_actions = test_env.action_space.n
observation, _ = test_env.reset(None, options={'enable_random_pos': True})
state = int((observation['agent'][0] * grid_world_size) + observation['agent'][1])
done = False
total_reward = 0

while not done:
    action = np.argmax(q_table[state])
    next_state, reward, terminated, truncated, _ = test_env.step(action)
    state = int((next_state['agent'][0] * grid_world_size) + next_state['agent'][1])
    total_reward += reward
    done = terminated or truncated
    time.sleep(0.2)

print(f"测试 episode 总奖励: {total_reward}")

{'agent': array([443, 539]), 'target': array([322, 234])}


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices